In [248]:

import time
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from functools import reduce

### 1.数据的读取

In [249]:
# 注意，一般需要对于数据进行统一处理
train = pd.read_csv('../data/round2_train.txt', sep=' ',nrows=100)
testa = pd.read_csv('../data/round2_ijcai_18_test_a_20180425.txt', sep=' ',nrows=100)
testb = pd.read_csv('../data/round2_ijcai_18_test_b_20180510.txt', sep=' ',nrows=100)

### 2.数据的预处理
#### 2.1 为testa，b统一添加 is_trade 变量

In [250]:
testa['is_trade'] = -1
testb['is_trade'] = -2

#### 2.2 生成test 数据集和data全量数据集

In [273]:
test=pd.concat([testa,testb])
data = pd.concat([train, test]).reset_index(drop=True)

#### 2.3 日期处理

##### 2.3.1 时间戳处理函数

In [274]:

def today(x):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))


##### 2.3.2 日期和小时特征抽取



In [275]:

def getday(x):
    day=int(x.split(' ')[0].split('-')[-1])
    if day==31:
        day=0
    return day

def gethour(x):
    # 总的来说，这里的处理是把时间单位细化了，用48 ‘hour’来记录一天，
    # 所以对于比如 0:00 0:30 1:00 会记录为 0:00 1:00 2:00
    hour=int(x.split(' ')[1].split(':')[0])    #  从 2018-09-06 21:58:20 获得 21
    minute=int(x.split(' ')[1].split(':')[1])  #  从 2018-09-06 21:58:20 获得 58 
    minute=1 if minute>=30 else 0              #  后半个小时记加 1 
    return hour*2+minute                       

##### 2.3.3 提取日期和时间信息

In [276]:
data = pd.concat([train, test]).reset_index(drop=True)
data['day']=data['context_timestamp'].apply(lambda x:getday(today(x))) # 提取了日期信息
data['hour']=data['context_timestamp'].apply(lambda x:int(today(x).split()[1].split(':')[0])) # 提取了小时信息

data['context_timestamp']=data['context_timestamp'].apply(lambda x:today(x)) # 把时间戳转化为 2018-09-06 21:58:20
data['hour48']=data['context_timestamp'].apply(gethour) # 具体操作请看 gethour 注释

### 3 对query的预处理

#### 首先来展示下数据
- 类别：商品具有的类别 【注】这里的问题，类别是一个三级的结构，每个商品严格意义上只有一个类别
- 属性：商品具有的属性 
- 预测：认为商品是某些类别，并判断属性 【注】预测的类别不具有层级结构


- item_category_list     
```
'836752724084922533;8769426218101861255'
```
- item_property_list
```
'7344985833148694227;6611726991947724280;7839592306500064003;9148482949976129397;2891682021882519203;6491818071284064879;8402837029054975192;4573204630033800991;296270104965272366;5512096939518377359;720840888466250585;7813953447682596156;3108863080117742654;2408002633983396839;2886347676380617486;6467463580352992420;8876749174447494343;834709706843352941;7420970629504412536;5293683912990614211;1111963465991721248;6314021851828608197;3600371705120228770;7078696387843273342;688979592902776162'
```

- predict_category_property 
```
'8769426218101861255:-1;836752724084922533:3480876984427839465,5103206280374780290,1972069903722514407, 
2636395404473730413,3540471119339706981,6491818071284064879'
```

#### 处理1：判断预测的类别和真实类别重合的个数

In [277]:
def same_cate(x):
    cate = set(x['item_category_list'].split(';'))                                            # 首先提取特征
    cate2 = set([i.split(':')[0] for i in x['predict_category_property'].split(';')])         # 其次提取预测的特征
    return len(cate & cate2)

data['same_cate']=data.apply(same_cate,axis=1) #相同类别数

In [278]:
data.item_category_list.head().values[:3]

array(['836752724084922533;8769426218101861255',
       '836752724084922533;6670526099037031245',
       '836752724084922533;6693726201323251689'], dtype=object)

#### 处理2：提取预测的属性和真实属性的重合个数

In [279]:
def same_property(x):
    
    property_a = set(x['item_property_list'].split(';')) # 获取属性
    
    # 获得属性比较困难，因为有的类别的属性为 -1
    # predict 字段的形式
    # '8769426218101861255:-1;836752724084922533:3480876984427839465,5103206280374780290,1972069903722514407, 2636395404473730413,3540471119339706981,6491818071284064879'
    
    # property_b : 拿到对商品预测的所有属性的去重结果 
    a = []
    for i in [i.split(':')[1].split(',') for i in x['predict_category_property'].split(';') if
              len(i.split(':')) > 1]:
        a += i
    property_b = set(a)
    
    # 返回预测中的属性的个数
    return len(property_a & property_b)

data['same_property']=data.apply(same_property,axis=1) #相同属性数

#### 处理3  获取商品的属性数目

In [280]:
data['property_num']=data['item_property_list'].apply(lambda x:len(x.split(';'))) #属性的数目

#### 处理4 获取query的类别的数目

In [281]:
data['pred_cate_num']=data['predict_category_property'].apply(lambda x:len(x.split(';'))) #query的类别数目

#### 处理5  query的属性数目

In [282]:

def f(x):
    try:
        return len([i for i in reduce((lambda x, y: x + y), [i.split(':')[1].split(',') for i in x.split(';') if len(i.split(':'))>1]) if i != '-1'])
    except:
        return 0

data['pred_prop_num']=data['predict_category_property'].apply(f) #query的属性数目



##### 解释一下

In [283]:


# 解释一下：
predict = '8769426218101861255:-1;836752724084922533:3480876984427839465,5103206280374780290,1972069903722514407, 2636395404473730413,3540471119339706981,6491818071284064879'
print("1 在 \n [i.split(':')[1].split(',') for i in predict.split(';') if len(i.split(':'))>1]  \n 这一步获取了每个类别预测的属性数 : \n",[i.split(':')[1].split(',') for i in predict.split(';') if len(i.split(':'))>1])
print("2 在 \n" ," [i for i in reduce((lambda x, y: x + y), [i.split(':')[1].split(',') for i in predict.split(';') if len(i.split(':'))>1]  ) if i != '-1'] \n 这一步把每个类目下预测的属性的list串联了起来 : \n" , [i for i in reduce((lambda x, y: x + y), [i.split(':')[1].split(',') for i in predict.split(';') if len(i.split(':'))>1]  ) if i != '-1']  )
print("3 然后用len 统计了长度")

1 在 
 [i.split(':')[1].split(',') for i in predict.split(';') if len(i.split(':'))>1]  
 这一步获取了每个类别预测的属性数 : 
 [['-1'], ['3480876984427839465', '5103206280374780290', '1972069903722514407', ' 2636395404473730413', '3540471119339706981', '6491818071284064879']]
2 在 
  [i for i in reduce((lambda x, y: x + y), [i.split(':')[1].split(',') for i in predict.split(';') if len(i.split(':'))>1]  ) if i != '-1'] 
 这一步把每个类目下预测的属性的list串联了起来 : 
 ['3480876984427839465', '5103206280374780290', '1972069903722514407', ' 2636395404473730413', '3540471119339706981', '6491818071284064879']
3 然后用len 统计了长度


#### query的第一个类别和所有类别

In [284]:
data['query1']=data['predict_category_property'].apply(lambda x:x.split(';')[0].split(':')[0]) #query第一个类别
data.query1.values[0]

'8769426218101861255'

#### query的所有类别

In [285]:
data['query']=data['predict_category_property'].apply(lambda x:'-'.join(sorted([i.split(':')[0] for i in [i for i in x.split(';')]]))) #query的全部类别
data['query'].values[:2]

array(['836752724084922533-8769426218101861255',
       '1852600517265062354-6670526099037031245-836752724084922533'],
      dtype=object)

### 4 对category 的处理

#### 提取二级标签

In [286]:
data['cate'] = data['item_category_list'].apply(lambda x: x.split(';')[1])
data.item_category_list.head() # 注意到一级标签都是接近的,且注意到第二条数据二级类目和 query 结果一致

0               836752724084922533;8769426218101861255
1               836752724084922533;6670526099037031245
2               836752724084922533;6693726201323251689
3               836752724084922533;6670526099037031245
4    836752724084922533;3613783563199627217;6370392...
Name: item_category_list, dtype: object

In [287]:
data.info(0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 38 columns):
instance_id                  300 non-null int64
item_id                      300 non-null int64
item_category_list           300 non-null object
item_property_list           300 non-null object
item_brand_id                300 non-null int64
item_city_id                 300 non-null int64
item_price_level             300 non-null int64
item_sales_level             300 non-null int64
item_collected_level         300 non-null int64
item_pv_level                300 non-null int64
user_id                      300 non-null int64
user_gender_id               300 non-null int64
user_age_level               300 non-null int64
user_occupation_id           300 non-null int64
user_star_level              300 non-null int64
context_id                   300 non-null int64
context_timestamp            300 non-null object
context_page_id              300 non-null int64
predict_category_property   

### 5 缺失值处理

In [288]:
def fillna(data):
    # 对数值变量 和 字符变量 做处理 
    numeric_feature = ['day', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level',
                       'user_age_level', 'user_star_level', 'shop_review_num_level',
                       'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 'shop_score_delivery',
                       'shop_score_description', 'context_page_id'
                       ]
    # 字符型数据
    string_feature = ['shop_id', 'item_id', 'user_id', 'item_brand_id', 'item_city_id', 'user_gender_id',
                      'user_occupation_id', 'context_page_id', 'hour']
    other_feature = ['item_property_list', 'predict_category_property']
    
    #填充缺失值
    # 这道题的关键：缺失值处的值为 -1 
    for i in string_feature+other_feature:
        # 使用众数填充缺失值
        mode_num = data[i].mode()[0]
        if (mode_num != -1):
            print(i)
            data.loc[data[i] == -1, i] = mode_num
        else:
            print(-1)
    for i in numeric_feature:
        # 使用均值填充缺失值
        mean_num = data[i].mean()
        if (mean_num != -1):
            print(i)
            data.loc[data[i] == -1, i] = mean_num
        else:
            print(-1)
    return data
data=fillna(data.copy())

shop_id
item_id
user_id
item_brand_id
item_city_id
user_gender_id
user_occupation_id
context_page_id
hour
item_property_list
predict_category_property
day
item_price_level
item_sales_level
item_collected_level
item_pv_level
user_age_level
user_star_level
shop_review_num_level
shop_review_positive_rate
shop_star_level
shop_score_service
shop_score_delivery
shop_score_description
context_page_id


### 6 对property feature的抽取

In [289]:
def property_feature(org):
    org = org.copy()
    # 注意，这件事情只能对property 做， category 是有级联关系的
    # 拿到所有的 item 属性信息 ， 注意是对于所有的样本的所有的属性信息
    tmp=org['item_property_list'].apply(lambda x:x.split(';')).values
    
    # 拿到属性信息（字典）
    property_dict={}
    # 拿到属性信息 （list）
    property_list=[]
    
    # 将所有的property_list 串联起来
    for i in tmp:
        property_list+=i
    
    # 统计所有出现过的property
    for i in property_list:
        if i in property_dict:
            property_dict[i]+=1
        else:
            property_dict[i] = 1
    
    print('dict finish')
    
    def top(x):
        # 对每条数据 ， 拆分出它的property 
        propertys=x.split(';')
        
        # 然后查找他所具有的property 在所有出现过的property 中的出现次数
        cnt=[property_dict[i] for i in propertys]
        
        # 根据出现次数进行排序， 注意这里对性质利用的方法
        res=sorted(zip(propertys,cnt),key=lambda x:x[1],reverse=True)
        # top1 是其所具有的最受欢迎的property 
        top1=res[0][0]
        # top2-10 是其所具有的最受欢迎的几个property的组合
        top2 = '_'.join([i[0] for i in res[:2]])
        top3 = '_'.join([i[0] for i in res[:3]])
        top4 = '_'.join([i[0] for i in res[:4]])
        top5='_'.join([i[0] for i in res[:5]])
        top10 = '_'.join([i[0] for i in res[:10]])
        return (top1,top2,top3,top4,top5,top10)
    # 对每个类别，用他所最受欢迎的property来表示property
    org['top']=org['item_property_list'].apply(top)
    print('top finish')
    org['top1']=org['top'].apply(lambda x:x[0])
    org['top2'] = org['top'].apply(lambda x: x[1])
    org['top3'] = org['top'].apply(lambda x: x[2])
    org['top4'] = org['top'].apply(lambda x: x[3])
    org['top5'] = org['top'].apply(lambda x: x[4])
    org['top10'] = org['top'].apply(lambda x: x[5])
    return org[['instance_id','top1','top2','top3','top4','top5','top10']]


# property_feature(data).info()
data=pd.merge(data,property_feature(data),on='instance_id',how='left') #拆分属性

dict finish
top finish


### 完成编码

In [291]:
#类别特征全部编码
def encode(data):
    id_features=['shop_id', 'item_id', 'user_id', 'item_brand_id', 'item_city_id', 'user_gender_id','item_property_list', 'predict_category_property',
                      'user_occupation_id', 'context_page_id','top1','top2','top3','top4','top5','top10','query1','query','cate']
    for feature in id_features:
        data[feature] = LabelEncoder().fit_transform(data[feature])
    return data

data=encode(data)